# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init("/home/rodolfo/spark-3.3.1-bin-hadoop3")
from pyspark.sql import SparkSession

In [2]:
from pyspark.ml.regression import LinearRegression
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

In [3]:
spark = SparkSession.builder.appName("lr").getOrCreate()

22/11/30 21:34:55 WARN Utils: Your hostname, rodolfo-300E5M-300E5L resolves to a loopback address: 127.0.1.1; using 192.168.15.11 instead (on interface wlp3s0)
22/11/30 21:34:55 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/11/30 21:34:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/11/30 21:34:57 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [4]:
df = spark.read.csv("../../data/cruise_ship_info.csv", header=True, inferSchema=True)

In [6]:
df.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [5]:
df.show(3)

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+
only showing top 3 rows



In [13]:
df.select(["Cruise_line", "crew"]).groupBy("Cruise_line").mean().show()

+-----------------+------------------+
|      Cruise_line|         avg(crew)|
+-----------------+------------------+
|            Costa|               7.7|
|              P&O| 8.588333333333333|
|           Cunard|10.246666666666668|
|Regent_Seven_Seas|             3.146|
|              MSC|              7.54|
|         Carnival| 10.13590909090909|
|          Crystal|             5.905|
|           Orient|               3.5|
|         Princess|  9.32235294117647|
|        Silversea|            2.4725|
|         Seabourn|1.6000000000000003|
| Holland_American| 6.215000000000002|
|         Windstar|1.1866666666666668|
|           Disney|              9.45|
|        Norwegian| 8.087692307692308|
|          Oceania|               4.0|
|          Azamara|              3.55|
|        Celebrity|             8.072|
|             Star| 5.363333333333333|
|  Royal_Caribbean|10.016521739130434|
+-----------------+------------------+



In [14]:
from pyspark.ml.feature import StringIndexer

In [15]:
indexer = StringIndexer(
    inputCol="Cruise_line",
    outputCol="Cruise_cat"
).fit(df)

In [17]:
indexer.transform(df).show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|Cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

In [22]:
assembler = VectorAssembler(
    inputCols=[
        "Age",
        "Tonnage",
        "passengers",
        "length",
        "cabins",
        "passenger_density",
        "Cruise_cat"
    ],
    outputCol="features"
)

In [25]:
df_preprocessed = assembler.transform(
    indexer.transform(df)
).select(["features", "crew"])

In [27]:
df_preprocessed.show(2)

+--------------------+----+
|            features|crew|
+--------------------+----+
|[6.0,30.276999999...|3.55|
|[6.0,30.276999999...|3.55|
+--------------------+----+
only showing top 2 rows



In [28]:
train, test = df_preprocessed.randomSplit([0.7, 0.3])

In [30]:
lr = LinearRegression(
    labelCol="crew"
).fit(train)

22/11/30 21:43:13 WARN Instrumentation: [685a0a38] regParam is zero, which might cause numerical instability and overfitting.


In [31]:
test_resuts = lr.evaluate(test)

In [32]:
test_resuts.meanAbsoluteError

0.7224384759285671

In [33]:
test_resuts.r2

0.9180497151168833